# **GEOG 60.01** ***Practical 4: Experimental Design and FaIR***

   <center><img src=https://docs.fairmodel.net/en/latest/_images/outline.png width="600"></center>

From: FaIR Model

## 0. Recap:

Where are we? Let's put together the pieces:

0. We've thought about the Earth as a **system** wanting to acheive balance or **equilibrium**, just like a bathtub, through a **negative feedback**:

$$\text{IN} = \text{OUT}$$

1. We've learned that the Earth acheives this balance by absorbing shortwave light from the sun and radiating longwave infrared light to space based on its temperature, all consistent with **Stefan-Boltzmann** and $\sigma T^4$ — a negative feedback, called the **Planck Feedback**. 

2. We learned that this equilibrium is actually aided by **convection**, a **choatic processes** (i.e., a sensitive dependence on **initial conditions**) that arises because of the differential heating of our planet and because of the vertical structure of our atmosphere. This is part of a wider process called **radiative-convective equilibrium**, which itself is shaped by feedbacks, such as the negative **lapse rate** feedback. So we learned a bit of atmospheric dynamics: differential heating induces differential densities inducing differential pressures, which gets air moving, as high pressure wants to equilibriate with low pressure.

3. We learned that simulating a system is hard because we can't do calculus on a computer (so errors are baked in), and in particular, simulating processes like convection is hard, first because is we don't have perfect measurements everywhere at every instant, and because computers have finite memory, and because the length scales and temporal scales of convection are small, making it incredibly computationally expensive to simulate (halving our spatial resolution ($\Delta X$) forces us to halve our timestep ($\Delta t$), making the cost $2^4=16$ more expensive.

4. We learned that climate models deal with this by **parameterizing** processes that occur on spatiotemporal scales smaller than a **grid box**, or a unit volume in a climate model that closes the **energy, mass, and momentum budgets**. These **unresolved** parameterizations, like our runoff module, influence the **resolved** processes in climate models, so the fine-scale influences the large-scale. There are a host of feedbacks within Earth's climate system, and the goal of the models is to get all of these processes and feedbacks represented to make predictions conditioned on a set of assumptions (or what we call a **projection**), such as what happens if we increase $\text{CO}_2$ at a rate of 1% per year. 

5. We've read that there are lots of [**IPCC**](https://www.ipcc.ch/) models out there, built by national labs around the world, all coordinating with each other to perfrom the same set of experiments, under the aegis of a program called the **Coupled Model Intercomparison Project**, which allows us to compare all the answers from different models given the same experiment. 

6. We've read that there are modeling best practices for evaluating the performance of these models and experiments, in particular, their representation of these key feedbacks, which ultimately determine what the temperature response of the planet is.

Now that we're caught up...


---
## 1. Goals of this practical

1. Learn about **climate sensitivity**

2. Learn to run a simple climate model, called FaIR

3. Apply some Xarray and plotting techniques
---

## 2. Climate Sensitivity

So we've got all these climate models, simulating the conservation of energy, mass, and momentum, all running the same coordinated experiments. How best to compare them?

One of the first things climate scientists want to compare among models is a **diagnostic** called [**Climate Sensitivity**](https://climate.mit.edu/explainers/climate-sensitivity).

The power of climate sensitivity is that it can be estimated from multiple sources -- observations, paleoclimate data, climate models, etc.

In particular, we're interested in two measures of climate senstivity:

- Equilibrium Climate Sensitivity, or ECS, defined as the equilibrium temperature response to a doubling of $\text{CO}_2$

and 

- Transient Climate Response, or TCR, defined as the mean global warming predicted to occur around the time of doubling $\text{CO}_2$

This idea of a doubling of $\text{CO}_2$ emerges because of the [band saturation effect](http://forecast.uchicago.edu/chapter4.pdf), which you may (or more may not) recall from your Archer reading. A brief explanation on why the band saturation and ECS are related. To the whiteboard!

<center><img src=https://skepticalscience.com/graphics/Climate_Sensitivity_500.jpg width="600"> 
<div>
   From: Skeptical Science

We'll formalize some more concepts, below.

But first, let's look at our tool, the **Finite Amplitude Impluse Response (FaIR) Model**:

---
## 3. A Climate Emulator 
...let's consider a simple climate model. We need simple climate models because full complexity **Earth System Models (ESMs)** that are used in climate change projections such as those used by the IPCC are expensive to run. 

[FaIR, or the Finite Amplitude Impluse-Response Model](https://docs.fairmodel.net/en/latest/intro.html), is designed to emulate or mimic the behavior of more complex ESMs. The input parameters to FaIR can then be varied to assess the responses in the full range of uncertainty due to the carbon cycle, radiative forcing, and temperature response.

FaIR essentially does two things: (1) it converts emissions of greenhouse gases to a concentration and radiative forcing time series and (2) it takes that time series and translates it a global temperature change time series.

---

In [ ]:
# import python modules
import os
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.colors as colors

In [ ]:
# import FaIR version 2.1.0
#https://docs.fairmodel.net/en/latest/intro.html

import fair
from fair import FAIR
from fair.io import read_properties
from fair.interface import fill, initialise

In [ ]:
# create FaIR instance
f = FAIR()

This `FAIR()` function creates an "instance" of the FaIR model in python, we've saved it in an object called "f" above. We can then use the object's built-in methods to define the terms of the simulation.

The vast majority of this simulation will be set up for you already, but let's try defining the timestep and time horizon of the experiment.

Let's use the `define_time()` function below. The function takes 3 arguments in order: beginning time, end time, and length of timestep (all units are years). For example, the code `f.define_time(0,50,0.5)` would run a simulation for 50 years, at 0.5-year timesteps.

**Try it below!** (Hint: pick a number of years that you think will be long enough for the climate system to reach equilibrium. You can re-run the below cells as many times as you need!)

You can check the times you generated by printing `f.timebounds` in a new cell if necessary. 

---
## 4. Equilibrium Climate Sensitivity
---

We're going to use FaIR to estimate ECS.

> The equilibrium climate sensitivity (ECS) is the long-term equilibrium global mean surface temperature response to a doubling of pre-industrial atmospheric $\text{CO}_2$ concentrations with all other forcers fixed at pre-industrial. It is a measure of long-term sensitivity of climate.

<center><img src=https://andthentheresphysics.files.wordpress.com/2017/09/knutti_etal.jpg width="600"></center>
<div>
Adapted from Knutti et al. 2017


ECS, and all climate scenarios rely on a concept called **radiative forcing**, which is defined as the change in the energy balance of Earth due to some factor that "forced" the change, hence the term "forcing".

Formally, we define the imbalance in the top of the atmosphere energy budget as in $\text{W m}^{-2}$:

$$ \Delta N = \Delta (\text{IN} - \text{OUT}) = \Delta \text{IN} - \Delta \text{OUT}$$

We can say that this imbalance is due to some perturbation to our system, aka a forcing, $\Delta F$, and then how that warming from that forcing and temperature-dependent feedbacks, like the ice-albedo feedback, or the water vapor feedback, or the lapse rate feedback, etc. shapes the energy emission back to space. There are a lot of feedbacks, and you may recall from systems theory, they can be positive (amplifiers) or negative (dampeners) of the perturbation.

We call the sum of all the feedbacks, $\lambda$, or the **climate feedback parameter**, which is just a number, it's a way to parameterize all the effects of all of those feedbacks. It's in units of $\text{W m}^{-2}\text{K}^{-1}$

Formally, we write all this as:

$$ \Delta N = \Delta F + \lambda\Delta T$$

So the above equation says we have a net imbalance (i.e., $\text{IN}\neq \text{OUT}$), which can be estimated as the the forcing, plus any warming dependent feedbacks in the climate system. 

At equilibrium, $\Delta N = 0$, because $\text{IN}=\text{OUT}$ again.

Therefore we can write this as:

$$\Delta T = \frac{-\Delta F}{\lambda}$$

Essentially, if I apply a forcing of $\Delta F$, then temperature increases by $\frac{F}{\lambda}$.

Because of this, ECS is defined as:

$$\text{ECS} = \Delta T_{2x} = \frac{-\Delta F_{2x}}{\lambda}$$

OK, let's run an ECS experiment using a 2xCO2 scenario in FaIR, where we instantaneously double the atmospheric concentration of $\text{CO}_2$.

The cell below will set up and run the rest of the simulation. You are not responsible for knowing everything, but we've included comments in case you are interested in seeing how the model works and potentially using it for a final project.

**TRY IT:** Run the cell after that to see a plot of temperature versus time, and adjust your time horizons accordingly if you don't think the sim reached equilibrium. This is essentially our most complex version of the bathtub model to-date!

In [ ]:
# Define scenarios (only one in this case)
f.define_scenarios(['abrupt-2xCO2'])

# Define configs (only doing one, default for now)
f.define_configs(['default'])

# Define species (and properties)

# in this case, only care about CO2 concentrations but model wants us to define nitrous oxide (N2O) and methane (CH4) as well
species = ['CO2', 'N2O', 'CH4']

# note that we set input_mode as concentration this time
properties = {
    'CO2': {
        'type': 'co2',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False,
    },
    'CH4': {
      'type': 'ch4',
      'input_mode': 'concentration',
      'greenhouse_gas': False, # note here
      'aerosol_chemistry_from_emissions': False,
      'aerosol_chemistry_from_concentration': True # we treat methane as a reactive gas (?)
    },
     'N2O': {
          'type': 'n2o',
          'input_mode': 'concentration',
          'greenhouse_gas': False, # note here (!!!)
          'aerosol_chemistry_from_emissions': False,
          'aerosol_chemistry_from_concentration': True # we treat nitrous oxide as a reactive gas (?)
     },
}


f.define_species(species, properties)

# create input and output data
f.allocate()

# fill in all data (initial conditions, configs, etc)

# this is filling 'concentration' from 'CO2' for our 'abrupt-2xCO2' scenario
# to ALWAYS equal 556.6 parts per million (which is double the measured pre-industrial levels of 278.3)
fill(f.concentration, 556.6, scenario='abrupt-2xCO2', specie='CO2') 

# Set CH4 and N20 to stay at their preindustrial levels
fill(f.concentration, 722e-3, scenario='abrupt-2xCO2', specie='CH4') # source: 722 ppb via wikipedia
fill(f.concentration, 275e-3, scenario='abrupt-2xCO2', specie='N2O') # source: Xu, et al. 2016 (https://cp.copernicus.org/articles/13/977/2017/cp-13-977-2017.pdf)

# Define first timestep
initialise(f.concentration, 278.3, specie='CO2')
initialise(f.concentration, 722e-3, specie='CH4')
initialise(f.concentration, 275e-3, specie='N2O')
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)

# fill all species configs as defaults
f.fill_species_configs()

# Fill climate configs -- these taken from the CENTRAL FaIR example configs (any built-in defaults?)
fill(f.climate_configs["ocean_heat_transfer"], [1.1, 1.6, 0.9])
fill(f.climate_configs["ocean_heat_capacity"], [8, 14, 100])
fill(f.climate_configs["deep_ocean_efficacy"], 1.1)

# finally, run FaIR
f.run(progress=False)

In [ ]:
# plot Global Mean Surface Temperature (GMST) time series
fig, ax = plt.subplots()

# plot surface (level 0) temp data for our 2xCO2 default run
ax.plot(f.timebounds, f.temperature.sel(scenario='abrupt-2xCO2', layer=0, config='default')) 
# note how f.temperature is an xr.DataArray, so we are selecting the appropriate dims above to plot.

ax.set(xlabel='YEAR', ylabel='TEMP. ANOMALY [K]', title='ECS SCENARIO ON FaIR DEFAULT')

**TRY IT:** You have now done a *graphical* solution of time until equilibrium, but can you write code below to compute the timestep at which the system reaches equilibrium? 

Let's assume equilibrium means a year-on-year change of less than 0.0001 Degrees Kelvin.

*Hint: Use the `times` and `temps` arrays defined for you to find the timestep where $(T_{time=i} - T_{time=i-1}) < 0.0001$*

In [ ]:
# 'times' and 'temps' arrays are defined for your convenience
times = f.timebounds
temps = f.temperature.sel(scenario='abrupt-2xCO2', layer=0, config='default')

**QUESTION:**

Based on your answer for time to equilibrium plus your knowledge of the computational costs of running Earth System Models like the ones in CMIP6, how do you think that we can get a sense of these models' ECS? 

**QUESTION:**

Based on the graph you generated above and the figure below, where does this instance of FaIR fall compared to the state-of-the-art CMIP6 Models with regard to ECS?

<center><img src=https://www.science.org/cms/10.1126/sciadv.aba1981/asset/a5916f25-f221-486d-9567-4f23d9d47dcb/assets/graphic/aba1981-f1.jpeg width="600"></center>
<div> From: Meehl, et al. (2020)

---
# 5. ECS Uncertainty from the Ocean
---

You may have noticed that in the code above we are running FaIR with 'default' parameters for things like `specie` (greenhouse gases species that impact the radiative balance of the planet) and climate configurations (things like how the ocean transfers and stores heat, which has a big impact on ECS and how long it takes to reach equilibrium as we learned in PS#1).

*There are tons more options for FaIR, but we won't be dealing with them today. However, there could be some cool final projects using FaIR with various configurations, or what FaIR calls* [configs](https://docs.fairmodel.net/en/latest/intro.html#adjusting-configs)...

You can learn more about the `climate configs` [here](https://docs.fairmodel.net/en/latest/api_reference.html#fair.energy_balance_model.EnergyBalanceModel)

Let's try running the same experiment as above, but with 3 separate ocean configurations from the FaIR website:

Below:

-`ocean_heat_capacity` (np.ndarray) – Ocean heat capacity of each layer (top first), in units of $\text{W m}^{-2} \text{yr K}^{-1}$

-`ocean_heat_transfer` (np.ndarray) – Heat exchange coefficient between ocean layers (top first). The first element of this array is akin to the climate feedback parameter, with the convention that stabilising feedbacks are positive (opposite to most climate sensitivity literature). In units of $\text{W m}^{-2} \text{yr K}^{-1}$

-`deep_ocean_efficacy` (float) – efficacy of deepest ocean layer. See e.g. [Geoffroy et al. 2013](https://doi-org.dartmouth.idm.oclc.org/10.1175/JCLI-D-12-00196.1).

All of these center on considering how the ocean shapes the rate of warming we actually experience from a perturbation like GHGs. For example, enhanced ocean heat uptake ***reduces*** the rate of warming and this effect occurs preferentially in some regions, such as those associated with regions of deep ocean water formation in the North Atlantic ocean and the circumpolar ocean of the Southern Hemisphere. Because the feedback strength varies geographically, the pattern of surface temperature change induced by the ocean heat uptake may impact the radiative imbalance, and therefore other feedbacks, $\lambda_{total} = \sum_{i=1}^{n}{\lambda_i}$, that influence ECS.

In [ ]:
# create FaIR instance
f = FAIR()

# define time horizon
f.define_time(0,1500,1)

# Define scenarios (only one in this case)
f.define_scenarios(['abrupt-2xCO2'])

# Define configs (THREE this time!)
fconfigs = ['high', 'central', 'low']
f.define_configs(fconfigs)

# Define species (and properties)

# in this case, only care about CO2 concentrations but model wants us to define nitrous oxide (N2O) and methane (CH4) as well
species = ['CO2', 'N2O', 'CH4']

# note that we set input_mode as concentration this time
properties = {
    'CO2': {
        'type': 'co2',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False,
    },
    'CH4': {
      'type': 'ch4',
      'input_mode': 'concentration',
      'greenhouse_gas': False, # note here
      'aerosol_chemistry_from_emissions': False,
      'aerosol_chemistry_from_concentration': True # we treat methane as a reactive gas (?)
    },
     'N2O': {
          'type': 'n2o',
          'input_mode': 'concentration',
          'greenhouse_gas': False, # note here (!!!)
          'aerosol_chemistry_from_emissions': False,
          'aerosol_chemistry_from_concentration': True # we treat nitrous oxide as a reactive gas (?)
     },
}


f.define_species(species, properties)

# create input and output data
f.allocate()

# fill in all data (initial conditions, configs, etc)

# this is filling 'concentration' from 'CO2' for our 'abrupt-2xCO2' scenario
# to ALWAYS equal 556.6 parts per million (which is double the measured pre-industrial levels of 278.3)
fill(f.concentration, 556.6, scenario='abrupt-2xCO2', specie='CO2') 

# Set CH4 and N20 to stay at their preindustrial levels
fill(f.concentration, 722e-3, scenario='abrupt-2xCO2', specie='CH4') # source: 722 ppb via wikipedia
fill(f.concentration, 275e-3, scenario='abrupt-2xCO2', specie='N2O') # source: Xu, et al. 2016 (https://cp.copernicus.org/articles/13/977/2017/cp-13-977-2017.pdf)

# Define first timestep
initialise(f.concentration, 278.3, specie='CO2')
initialise(f.concentration, 722e-3, specie='CH4')
initialise(f.concentration, 275e-3, specie='N2O')
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)

# fill all species configs as defaults
f.fill_species_configs()

# Fill climate configs -- these taken from the CENTRAL FaIR example configs (any built-in defaults?)
for cf in fconfigs:
    
    if cf=='low':
        # ocean heat transfer config
        oht = [1.7, 2.0, 1.1]
        # ocean heat capacity config
        ohc = [6, 11, 75]
        # deep ocean efficacy config
        doe = 0.8
    elif cf=='central':
        oht = [1.1, 1.6, 0.9]
        ohc = [8, 14, 100]
        doe = 1.1
    elif cf=='high':
        oht = [0.6, 1.3, 1.0]
        ohc = [5, 15, 80]
        doe = 1.29
        
    fill(f.climate_configs["ocean_heat_transfer"], oht, config=cf)
    fill(f.climate_configs["ocean_heat_capacity"], ohc, config=cf)
    fill(f.climate_configs["deep_ocean_efficacy"], doe, config=cf)
    
# finally, run FaIR
f.run(progress=False)

Look at `f.temperature`, you'll see that FaIR outputs data in Xarray format. 

What are the dimensions? Look back at `low`, `central` and `high`. What do you notice about the OHC, efficacy and transfer parameters? 

**TRY IT:** Let's try to adjust the plotting code from the previous section to plot global mean surface temperatures for the Low, Medium, and High configurations all on the same axes. The majority of the code is done for you, you just have to figure out how to select temperature data for the correct scenario, layer, and config.

Hint: Edit the command `f.temperature.sel()` with your selection criteria to grab the temps for each configuration.

**QUESTION:** 

What role do these configurations have on ECS? Do they matter?

---
# 6. Transient Climate Response (TCR) 
---

> The transient climate response (TCR) is the global mean surface temperature change at the point where double pre-industrial atmospheric CO2 concentration is reached in an experiment where $\text{CO}_2$ concentration is increased at a compound rate of 1% per year and all other forcers fixed at pre-industrial. This is around 69.7 years (many models use 70 years). It is a measure of both long-term sensitivity of climate and medium-term rate of climate response.

So caclulating the TCR requires a different **experimental design** than that for ECS. Instead of instantaneously doubling or quadrupling the $\text{CO}_2$ in the model, we're going to ramp it up more like we have been doing in the real world, say at a rate of a 1% increase per year, until we've doubled the amount of $\text{CO}_2$ in the atmosphere...

<center><img src=https://archive.ipcc.ch/ipccreports/tar/wg1/images/fig9-1.gif> </center>
From IPCC TAR WGI

Ok, so to do this, let's run FaIR at a constant annual rate of increase in $\text{CO}_2$ concentrations over pre-industrial levels. To do so, we first need to create a timeseries of $\text{CO}_2$ data. 

The most common design is to let the model "see" a 1% increase in atmospheric $\text{CO}_2$ concentrations per year until a doubling or quadrupling. We can borrow some simple logic from [compound interest](https://www.investor.gov/financial-tools-calculators/calculators/compound-interest-calculator), where we just have exponential growth in [$\text{CO}_2$].

**TRY IT:** Use your Python and NumPy skills to generate an array of $\text{CO}_2$ concentrations as inputs for this experiment. The array should be 140 elements long, with the initial value at 278.3 and a 1% increase at each step. Save the array to varname `co2_arr` so that it gets automatically plugged in to the FaIR code below.

*Hint: One way to represent this 1% exponential growth would be with the following equation:* $[CO_2] = 278.3 \cdot (1 + 0.01)^{t}$

Next, verify that your `co2_arr` looks as it should (exponential growth, final value around 1113.2) by plotting, indexing parts of the array, or both. `co2_arr.shape` should be (140,) for the FaIR code to work.

It's ok if your final value is greater than 113.2, because doubling only takes 69.7 years so a quadrupling will take less than 140!

Now let's create our FaIR instance. Define the time so that the simulation runs for 140 years, with a timestep representing 1 year. 

In [ ]:
# Create FaIR instance
f = FAIR()


# Define time horizon (want 140 years now)


Now we will run FaIR using the CO2 array we just created as the CO2 input forcings.

In [ ]:
# Define scenarios (only one in this case)
f.define_scenarios(['TCR'])

# Define configs (only doing one, default for now)
f.define_configs(['default'])
f.configs

# Define species (and properties)

# in this case, only specie is CO2 concentrations
species = ['CO2', 'N2O', 'CH4']

# note that we set input_mode as concentration this time
properties = {
    'CO2': {
        'type': 'co2',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False,
    },
    'CH4': {
      'type': 'ch4',
      'input_mode': 'concentration',
      'greenhouse_gas': False, # note here
      'aerosol_chemistry_from_emissions': False,
      'aerosol_chemistry_from_concentration': True # we treat methane as a reactive gas (?)
    },
     'N2O': {
          'type': 'n2o',
          'input_mode': 'concentration',
          'greenhouse_gas': False, # note here (!!!)
          'aerosol_chemistry_from_emissions': False,
          'aerosol_chemistry_from_concentration': True # we treat nitrous oxide as a reactive gas (?)
     },
}


f.define_species(species, properties)

# create input and output data
f.allocate()

# fill in all data (initial conditions, configs, etc)


co2_arr_forfill = np.expand_dims(co2_arr, axis=1) # just need to reshape the co2_arr
                                     # so it lines up with FaIR's desired dimensions

# this is filling 'concentration' from 'CO2' for our 'TCR' scenario
# to be the array that we came up with before
fill(f.concentration, co2_arr_forfill, scenario='TCR', specie='CO2') 

# PREINDUSTRIAL METHANE = 722 PPB = 722e-3 ppm (wikipedia)
# PREINDUSTRIAL N2O... 275 ppb = 275e-3 ppm  (Xu, et al. 2016 https://cp.copernicus.org/articles/13/977/2017/cp-13-977-2017.pdf)

# Set CH4 and N20...
fill(f.concentration, 722e-3, scenario='TCR', specie='CH4')
fill(f.concentration, 275e-3, scenario='TCR', specie='N2O')

# Define first timestep
initialise(f.concentration, 278.3, specie='CO2')
initialise(f.concentration, 722e-3, specie='CH4')
initialise(f.concentration, 275e-3, specie='N2O')
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

# fill all species configs as defaults
f.fill_species_configs()

# Fill climate configs -- these taken from the CENTRAL FaIR example configs (any built-in defaults?)
fill(f.climate_configs["ocean_heat_transfer"], [1.1, 1.6, 0.9])
fill(f.climate_configs["ocean_heat_capacity"], [8, 14, 100])
fill(f.climate_configs["deep_ocean_efficacy"], 1.1)

# run FaIR
f.run(progress=False)

... And, let's plot. 

In [ ]:
# plot
fig, ax = plt.subplots()

# plot surface (level 0) temp data for our TCR default run
ax.plot(f.timebounds, f.temperature.sel(scenario='TCR', layer=0, config='default')) 

# labels
ax.set(xlabel='YEAR', ylabel='TEMP. ANOMALY [K]', title='TCR SCENARIO ON FaIR DEFAULT')

*** QUESTION:*** Notice we put in exponential $\text{CO}_2$, but that temperature scales relatively linearly with it. Why?

Lastly, lets find the precise value of ∆T after 70 years of this simulation and compare that to the Meehl figure above. Does this square away with the CMIP6 TCR estimates?

---
# 7. Run a Bunch of SSP's with FaIR
---

Demonstration: FaIR also has lots of built-in [SSP (Shared Socioeconomic Pathway)](https://www.carbonbrief.org/explainer-how-shared-socioeconomic-pathways-explore-future-climate-change/) emissions scenarios that can be used to run the model. We will demonstrate this by running the available scenarios below. An interesting final project could include running some sort of FaIR experiments with these SSP's or modifying the scenarios to your liking...

<center><img src=https://climatedata.ca/site/assets/uploads/2022/03/SSPexplainer_Figure4.png></center>
From ClimateData.ca

In [ ]:
# init fair
f = FAIR(ch4_method='thornhill2021') #we want to enable the methane lifetime routine that is a function of SLCFs and reactive gases, see documentation
# set time horizon
f.define_time(1750, 2100, 1)

# Define SSP scenarios (these all come from RCMIP)
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585'] 
f.define_scenarios(scenarios)

# define configs - still just the one default
f.define_configs(['default'])

# define species and properties
species, properties = read_properties()
f.define_species(species, properties)

# allocate for input and output data
f.allocate()

# fill in the data 

# (get default species configs)
f.fill_species_configs()

# get emissions + solar and volcanic forcing from RCMIP datasets using 'fill_from_rcmip()' helper function
f.fill_from_rcmip()

# initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

# fill climate configs using default middle-of-the-road vals from above
fill(f.climate_configs["ocean_heat_transfer"], [1.1, 1.6, 0.9])
fill(f.climate_configs["ocean_heat_capacity"], [8, 14, 100])
fill(f.climate_configs["deep_ocean_efficacy"], 1.1)

# run FaIR
f.run(progress=False)

In [ ]:
# Plot temp timeseries for each of the scenarios

fig, ax = plt.subplots(figsize=(12, 6))

# plot each ssp
for ssp in scenarios:

    ax.plot(f.timebounds, f.temperature.sel(scenario=ssp, layer=0), label=ssp)
    
# titles / labels
ax.set(title='FaIR SSPs: GLOBAL MEAN SURFACE TEMPERATURE', 
       xlabel='YEAR', 
       ylabel='TEMPERATURE ANOMALY (K)')

# legend
ax.legend()

# look from 2000-2100
ax.set_xlim(2000,2100)

# line at present day
ax.axvline(2023, ls='--', color='grey')

**Play around with this -- the documentation is quite thorough and the possibilities with it are endless...**